## House Office Expenditure Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#import data and concatenate them into a single dataframe
head = 'data/'
tail = '-house-disburse-detail.csv'
first = pd.read_csv('data/2009Q3-house-disburse-detail.csv', encoding = "ISO-8859-1")
second = pd.read_csv('data/2009Q4-house-disburse-detail.csv', encoding = "ISO-8859-1")
last = pd.read_csv('data/2018Q1-house-disburse-detail.csv', encoding = "ISO-8859-1")

df = pd.concat([first,second], ignore_index = True)

for i in range(8):
    for j in range(4):
        file_name = head+str(2010+i)+'Q'+str(j+1)+tail
        current = pd.read_csv(file_name, encoding = "ISO-8859-1")
        df = pd.concat([df, current], ignore_index = True)
        
df.QUARTER = df.QUARTER.replace(to_replace = 'Q3', value = '2017Q3')
df.QUARTER = df.QUARTER.replace(to_replace = 'Q4', value = '2017Q4')
last['QUARTER'] = '2018Q1'
        
df = pd.concat([df, last], ignore_index = True)
df.shape

In [2]:
df.to_csv('House.csv')

/Users/Xuntao/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,4,8,9,11,12,13,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Answer to Question 1:
'What is the total of all the payments in the dataset?''

In [3]:
# Normalizing the AMOUNT column, to make them all float type
df['AMOUNT'] = df.AMOUNT.apply(lambda x: 0 if x == '#VALUE!' else x)
df.AMOUNT = df.AMOUNT.apply(lambda x: np.float64(str(x).replace(',','')))

#We compute the total sum. Note that we did not take away negative terms. Since that reflects return of flow of money or income.
total_sum = df.AMOUNT.sum()
total_sum

12699262210.909998

## Data Cleaning
Now we start cleaning the data, since this includes dropping some cells we don't want, so we have to compute the total amount before that

In [4]:
df.YEAR = df.YEAR.fillna(value = 0)
# Normalize the YEAR column
def year_normalizer(x):
    if type(x) == str:
        x = x.replace('FISCAL YEAR ', '')
        x = x.replace('#VALUE!', '0')
    return int(x)

df.YEAR = df.YEAR.apply(lambda x: year_normalizer(x))

In [5]:
# Normalize the OFFICE column
def office_normalizer(x):
    irr = ['FISCAL YEAR 2017 ', 'FISCAL YEAR 2016 ', 'FISCAL YEAR 2015 ', 'FISCAL YEAR 2018 ',
          'FISCAL YEAR 2014 ', 'FISCAL YEAR 2013 ', 'FISCAL YEAR 2012 ', '2012 ',
          '2013 ', '2014 ', '2015 ', '2016 ', '2017 ', '2018 ']
    for entry in irr:
        if entry in x:
            x = x.replace(entry, '')
    return x

df.OFFICE = df.OFFICE.apply(lambda x: office_normalizer(x))

In [6]:
# For the purpose of later questions, we need to distinguish offices of representatives from others.
def rep_distinguisher(x):
    if 'HON.' in x:
        return True
    else: return False

df['if_rep'] = df.OFFICE.apply(lambda x: rep_distinguisher(x))

# Store the data from representatives in df_rep
df_rep = df[df.if_rep == True]

In [7]:
# Drop the entries with empty start or end date since the number is small (<20)
df = df.dropna(axis = 0, subset = ['START DATE'])
df.reset_index(drop = True, inplace = True)

In [8]:
# Drop the anomalies (with the wrong format) in EndDate and StartDate
def check_date_format(x):
    if len(x)>11 or len(x)<5: return False
    elif any(c.isalpha() for c in x) == True: return False
    else: return True

# Create two columns to determine whether the dates are valid
df['StartDateCheck'] = df['START DATE'].apply(lambda x: check_date_format(x))
df['EndDateCheck'] = df['END DATE'].apply(lambda x: check_date_format(x))

len(df[df['StartDateCheck'] == True])

3515678

In [9]:
df = df[df['StartDateCheck'] == True]
df.reset_index(drop= True, inplace = True)

In [12]:
# Converting the start date to TimeStamp
df['START DATE'] = pd.to_datetime(df['START DATE'])

## Answer to Question 2:
Define the 'COVERAGE PERIOD' for each payment as the difference (in days) between 'END DATE' and 'START DATE'. What is the standard deviation in 'COVERAGE PERIOD'? Only consider payments with strictly positive amounts.

In [18]:
# Drop the entries with empty start or end date since the number is small (<20)
df_end = df.dropna(axis = 0, subset = ['START DATE'])
df_end.reset_index(drop = True, inplace = True)

# Take the one with valid end dates and change the end dates to TimeStamp
df_end = df_end[df_end['EndDateCheck'] == True]
df_end['END DATE'] = pd.to_datetime(df_end['END DATE'])

# pick the entries with positive amounts
df_pos_amount = df_end[df_end.AMOUNT > 0]
df_pos_amount.reset_index(drop = True, inplace = True)

In [19]:
# Computing the difference between End date and Start date
df_pos_amount['COVERAGE PERIOD']= (df_pos_amount['END DATE']-df_pos_amount['START DATE'])
df_pos_amount['COVERAGE PERIOD'] = df_pos_amount['COVERAGE PERIOD'].apply(lambda x: x.days)

# drop the one with negative coverage periods, there were 52 entries.
df_pos_amount = df_pos_amount[df_pos_amount['COVERAGE PERIOD']>=0]

# Find the standard Deviation
np.std(list(df_pos_amount['COVERAGE PERIOD']))

/Users/Xuntao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Xuntao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


60.12006319096592

## Answer to the question 3: 
What was the average annual expenditure with a 'START DATE' date between January 1, 2010 and December 31, 2016 (inclusive)? Only consider payments with strictly positive amounts.

In [13]:
# create a START YEAR column
df['START YEAR'] = df['START DATE'].apply(lambda x: x.year)

In [14]:
# select the rows with start year in 2011 to 2016
mask = (df['START YEAR']>=2010) & (df['START YEAR']<=2016)
df_inrange = df[mask]
df_inrange.reset_index(drop = True, inplace = True)

In [15]:
# pick the entries with positive amount
df_pos_amount_inrange = df_inrange[df_inrange.AMOUNT > 0]
df_pos_amount_inrange.reset_index(drop = True, inplace = True)

In [16]:
total_exp = df_pos_amount_inrange.AMOUNT.sum()

In [17]:
#Compute the average
ave = total_exp/7
ave

1219719497.8385723

## Answer to question 4: 
"Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. For this office, find the 'PURPOSE' that accounts for the highest total expenditures. What fraction of the total expenditures (all records, all offices) with a 'START DATE' in 2016 do these expenditures amount to?"

In [20]:
#Select the rows that has a start date in 2016
df_start_2016 = df[df['START YEAR'] == 2016]

In [23]:
#Find the 'OFFICE' with the highest total expenditures with a 'START DATE' in 2016. 
grouped = df_start_2016.groupby(by = ['OFFICE']).sum().sort_values(by = 'AMOUNT', ascending = False)
grouped.reset_index(inplace = True)
top_office = grouped.OFFICE[0]

'GOVERNMENT CONTRIBUTIONS'

In [24]:
#For this office, find the 'PURPOSE' that accounts for the highest total expenditures. 
top_purpose = df_start_2016[df_start_2016.OFFICE == top_office].groupby('PURPOSE').sum().sort_values(by = 'AMOUNT', ascending = False).AMOUNT[0]

#Compute the total expenditures in 2016
exp_2016 = df_start_2016.AMOUNT.sum()

#Compute the fraction.
top_purpose / exp_2016

0.06991225136352171

## Answer to the Question 5:
What was the highest average staff salary among all representatives in 2016? Assume staff sizes is equal to the number of unique payees in the 'PERSONNEL COMPENSATION' category for each representative.

In [7]:
# Pick the rows in YEAR 2016 (Note that this is different than START YEAR being 2016!) in df_rep
df_2016= df_rep[df_rep.YEAR == 2016]

# Look at only the expense in 'PERSONNEL COMPENSATION' category
df_2016_staff = df_2016[df_2016.CATEGORY == 'PERSONNEL COMPENSATION']

# Find out how many unique payees each office have in 2016, regard them as staff numbers of each office
df_2016_staff_number = df_2016_staff.groupby('OFFICE').PAYEE.nunique().to_frame()

# Summing up the expenses in the "Personnel compensation" category (salary) in each office
df_2016_expense = df_2016_staff.groupby('OFFICE').sum()

# Concatenate the two dataframes
df_2016_staff_expense = pd.concat([df_2016_expense, df_2016_staff_number], axis = 1)

# Compute the average in each office
df_2016_staff_expense['AVERAGE'] = df_2016_staff_expense.AMOUNT/df_2016_staff_expense.PAYEE

# Pick the top one
df_2016_staff_expense.sort_values('AVERAGE', ascending = False).AVERAGE[0]

34575.83428571429

## Answer to the question 6:
What was the median rate of annual turnover in staff between 2011 and 2016 (inclusive)? Turnover for 2011 should be calculated as the fraction of a representative's staff from 2010 who did not carry over to 2011. Only consider representatives who served for at least 4 years and had staff size of at least 5 every year that they served.

In [8]:
# Choose representative who served for at least 4 years and is currently in office in 10-16.
# We need to use data from 2010 since we need that to compute the turnovers in 2011
senior_reps = df_rep.groupby('OFFICE').filter(lambda x: x.max()['YEAR']-x.min()['YEAR']>=4)
senior_reps = senior_reps[senior_reps.YEAR >=2010]
senior_reps = senior_reps[senior_reps.YEAR <=2016]

# Choose representative that has at least 5 staffs every year that they served.
senior_reps = senior_reps[senior_reps.CATEGORY == 'PERSONNEL COMPENSATION']
senior_reps.dropna(axis = 0, subset = ['PAYEE'])
senior_reps = senior_reps.groupby(by = ['OFFICE','YEAR']).filter(lambda x: x.PAYEE.nunique()>=5)

# return a dataframe with list of employees in each office in each year
unique_employees = senior_reps.groupby(by = ['OFFICE','YEAR']).PAYEE.unique()
unique_employees = unique_employees.reset_index()

# Return a list of the turnover rates in all reps offices in the given period.
reps = list(unique_employees.OFFICE.unique())
turnover_rate = []
for rep in reps:
    this_rep = unique_employees[unique_employees.OFFICE == rep]
    this_rep.reset_index(drop=True, inplace = True)
    n = len(this_rep)-1
    for i in range(n):
        rate = len(set(this_rep.PAYEE[i])-set(this_rep.PAYEE[i+1]))/len(set(this_rep.PAYEE[i+1]))
        turnover_rate.append(rate)

# Compute the median for all turnover rates in 11-16
median = np.median(turnover_rate)

In [9]:
median

0.23076923076923078

## Answer to the question 7: 
What percentage of the expenditures of the top 20 spenders in 2016 come from members of the Democratic Party? Representatives are identified by their 'BIOGUIDE_ID', which can be used to look up representatives with ProPublica's Congress API to find their party affiliation. Consider an expenditure as being in 2016 if its 'START DATE' is in 2016.

In [25]:
# Look up the top 20 spenders in 2016. Compute the total expenditures they spend.
df_start_2016_rep = df_start_2016[df_start_2016.if_rep == True]
top_20_2016 = df_start_2016_rep.groupby(by = ['OFFICE']).sum().sort_values(by = 'AMOUNT', ascending = False)[:20]
top_20_2016.reset_index(inplace = True)
top_20_2016['if_Dem'] = ['True','False','True','False','False','False','True','True','False','True','True','False','True','True','False','True','True','True','False','False']
top_20_2016

,OFFICE,Unnamed: 0,AMOUNT,YEAR,if_rep,StartDateCheck,EndDateCheck,START YEAR,if_Dem
0,HON. PEDRO R. PIERLUISI,1962727229,1773125.94,1376898,683.0,683.0,683.0,1376928,True
1,HON. EARL L. BUDDY CARTER,3273480326,1537780.51,2308301,1145.0,1145.0,1145.0,2308320,False
2,HON. CAROLYN B. MALONEY,2193515851,1409965.17,1507946,760.0,760.0,748.0,1532160,True
3,HON. RYAN K. ZINKE,3403977247,1360360.16,2326449,1163.0,1163.0,1154.0,2344608,False
4,HON. TED LIEU,2371672700,1357517.48,1626894,818.0,818.0,807.0,1649088,False
5,HON. WILLIAM LACY CLAY,1933759332,1343129.49,1328541,674.0,674.0,659.0,1358784,False
6,HON. JERROLD NADLER,1185985582,1340387.65,812436,410.0,410.0,403.0,826560,True
7,HON. BRUCE POLIQUIN,2316577350,1338873.00,1606732,798.0,798.0,797.0,1608768,True
8,HON. DOUG LAMALFA,2247794820,1338552.55,1558353,778.0,778.0,773.0,1568448,False
9,HON. MARTHA MCSALLY,2082605369,1338452.13,1453531,722.0,722.0,721.0,1455552,True


In [26]:
# Compute the expenditures spent by the Democratic members in the top 20 spenders in 2016.
tot_exp_2016_top_20_dem = top_20_2016[top_20_2016.if_Dem == 'True'].sum()['AMOUNT']
tot_exp_2016_top_20 = top_20_2016.sum()['AMOUNT']
tot_exp_2016_top_20_dem/tot_exp_2016_top_20

0.5533902011196373